<a href="https://colab.research.google.com/github/BuxGuerra/TPICD/blob/main/analiseFinalista.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
import pandas as pd
import numpy as np
from sklearn.metrics import (confusion_matrix, accuracy_score)
from sklearn.linear_model import LogisticRegression

In [122]:
data = pd.read_csv('https://raw.githubusercontent.com/BuxGuerra/TPICD/main/data/basesUnificadas/TemporadasUnificadas.csv')
data

,Unnamed: 0.1,Unnamed: 0,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,...,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,FINAL
0,0,0,1610612737,Atlanta Hawks,82,48,34,0.585,48.4,38.6,...,2,22,3,5,14,7,29,12,7,0.0
1,1,1,1610612738,Boston Celtics,82,48,34,0.585,48.2,39.2,...,6,12,2,22,24,29,10,5,8,0.0
2,2,2,1610612751,Brooklyn Nets,82,21,61,0.256,48.2,38.2,...,15,19,19,25,18,2,28,26,28,0.0
3,3,3,1610612766,Charlotte Hornets,82,48,34,0.585,48.4,37.0,...,19,1,22,12,21,3,14,11,9,0.0
4,4,4,1610612741,Chicago Bulls,82,42,40,0.512,48.5,38.6,...,12,13,29,8,27,5,26,21,18,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,25,25,1610612758,Sacramento Kings,82,27,55,0.329,48.2,38.8,...,25,9,11,25,14,19,30,30,28,0.0
386,26,26,1610612759,San Antonio Spurs,82,47,35,0.573,48.1,39.0,...,15,4,14,4,2,1,18,27,8,0.0
387,27,27,1610612761,Toronto Raptors,82,59,23,0.720,48.4,41.3,...,6,6,18,2,18,27,16,4,2,0.0
388,28,28,1610612762,Utah Jazz,82,48,34,0.585,48.2,38.3,...,21,20,4,9,11,16,8,19,5,0.0


In [115]:
data.isnull().sum()

Unnamed: 0.1       0
Unnamed: 0         0
TEAM_ID            0
TEAM_NAME          0
GP                 0
W                  0
L                  0
W_PCT              0
MIN                0
FGM                0
FGA                0
FG_PCT             0
FG3M               0
FG3A               0
FG3_PCT            0
FTM                0
FTA                0
FT_PCT             0
OREB               0
DREB               0
REB                0
AST                0
TOV                0
STL                0
BLK                0
BLKA               0
PF                 0
PFD                0
PTS                0
PLUS_MINUS         0
GP_RANK            0
W_RANK             0
L_RANK             0
W_PCT_RANK         0
MIN_RANK           0
FGM_RANK           0
FGA_RANK           0
FG_PCT_RANK        0
FG3M_RANK          0
FG3A_RANK          0
FG3_PCT_RANK       0
FTM_RANK           0
FTA_RANK           0
FT_PCT_RANK        0
OREB_RANK          0
DREB_RANK          0
REB_RANK           0
AST_RANK     

Análise com as estatísticas:

In [116]:
df = data

y = df['FINAL']
x = df.iloc[:,7:30]

logReg = LogisticRegression(max_iter=1000)
logReg.fit(x,y)

predicts = logReg.predict(x)
confusion_matrix(y,predicts)

array([[361,   3],
       [ 18,   8]])

Análise com todos os ranks:

In [117]:
df = data
df = df.iloc[:,33:]
x = df.iloc[:,:23]
y = df['FINAL']

logReg2 = LogisticRegression(max_iter=1000)
logReg2.fit(x,y)

predicts2 = logReg2.predict(x)
confusion_matrix(y,predicts2)

array([[362,   2],
       [ 14,  12]])

Análise com os ranks mais importantes de acordo com o outro colab:

In [118]:
df = data
df = df.iloc[:,30:]
x = df[['PLUS_MINUS_RANK','W_RANK','BLKA_RANK','FG_PCT_RANK','DREB_RANK','FGM_RANK','AST_RANK','FG3_PCT_RANK','BLK_RANK']]
y = df['FINAL']

logReg3 = LogisticRegression(max_iter=1000)
logReg3.fit(x,y)

predicts3 = logReg3.predict(x)
confusion_matrix(y,predicts3)

array([[360,   4],
       [ 20,   6]])